In [1]:
# import numpy as np
import cvxpy as cp
import networkx as nx
import autograd.numpy as np
from autograd import grad

import pymanopt
import pymanopt.manifolds
import pymanopt.solvers

from numpy.random import default_rng
from complex_elliptope import ComplexElliptope

from opt_utils import decompose_psd, normalize_rows, complex_hyperplane_rounding

In [2]:
def get_rand_complex(m, n, max_val=5):
    return np.round(np.random.rand(m, n) * max_val) + np.round(np.random.rand(m, n) * max_val) * 1j

# Phase recovery

In [3]:
n = 50  # number of observations
p = 20  # dimension of x
max_val = 10

In [4]:
rng = default_rng()
A = rng.random((n,p)) * max_val + rng.random((n,p)) * max_val * 1j
assert np.linalg.matrix_rank(A, tol=1e-6) >= p  # A must be injective; if the rows of A are linearly independent, AA+ = I
b = rng.random(n) * max_val
M = np.diag(b) @ (np.identity(n) - A @ np.linalg.pinv(A)) @ np.diag(b)

\begin{equation*}
\begin{aligned}
\min \quad & u^{*}Mu \\
\textrm{s.t.} \quad & |u_{i}| = 1, \qquad i=1,...,n
\end{aligned}
\end{equation*}

In [5]:
# u = cp.Variable(n, complex=True)
# qp_constraints = [ cp.square(cp.real(u[i])) + cp.square(cp.imag(u[i])) == 1 for i in range(n) ]
# prob = cp.Problem(cp.Minimize(cp.quad_form(u, M)), qp_constraints)
# prob.solve()

\begin{equation*}
\begin{aligned}
\min \quad & M \cdot U \\
\textrm{s.t.} \quad & U_{ii} = 1, \qquad i=1,...,n \\
& U \succeq 0
\end{aligned}
\end{equation*}

### solve SDP with cvxpy

In [6]:
U = cp.Variable((n,n), hermitian=True)
# U = cp.Variable((n,n), symmetric=True)  # for testing

In [7]:
sdp_constraints = [U >> 0]
sdp_constraints += [U[i][i] == 1 for i in range(n)]

In [8]:
prob = cp.Problem(cp.Maximize(cp.real(cp.trace(M @ U))), sdp_constraints)
prob.solve()

print("The optimal value is", prob.value)
print("A solution U is", U.value)

The optimal value is 1515.6610902013365
A solution U is [[ 1.00000000e+00+0.j          7.53784948e-02+0.0081514j
  -1.63808256e-01+0.23070046j ... -4.31110044e-01-0.1728112j
   6.39486193e-03+0.19008847j -4.73541962e-02+0.6106984j ]
 [ 7.53784948e-02-0.0081514j   1.00000000e+00+0.j
  -7.00586728e-02-0.01875015j ... -6.35562352e-02-0.05255963j
  -5.71961459e-02-0.06798054j -1.41074042e-01-0.02366289j]
 [-1.63808256e-01-0.23070046j -7.00586728e-02+0.01875015j
   1.00000000e+00+0.j         ... -1.76185799e-01+0.23597661j
   1.77009731e-01-0.12711279j  3.16952149e-01-0.1005304j ]
 ...
 [-4.31110044e-01+0.1728112j  -6.35562352e-02+0.05255963j
  -1.76185799e-01-0.23597661j ...  1.00000000e+00+0.j
   1.98123476e-01+0.13683914j -8.41139112e-02-0.49046139j]
 [ 6.39486193e-03-0.19008847j -5.71961459e-02+0.06798054j
   1.77009731e-01+0.12711279j ...  1.98123476e-01-0.13683914j
   1.00000000e+00+0.j          4.03304182e-04-0.28553369j]
 [-4.73541962e-02-0.6106984j  -1.41074042e-01+0.02366289j
   3

In [9]:
opt_rank = np.linalg.matrix_rank(U.value, 1e-9)
print(opt_rank)

30


### Complex elliptope gradient

In [10]:
manifold = ComplexElliptope(n, opt_rank)

@pymanopt.function.autograd(manifold)
def manifold_cost(Y):
    return -np.real(np.trace(M @ Y @ np.conj(Y).T))

problem = pymanopt.Problem(manifold=manifold, cost=manifold_cost)
solver = pymanopt.solvers.SteepestDescent(minstepsize=1e-9)
solution = solver.solve(problem)

Optimizing...
Iteration    Cost                       Gradient norm     
---------    -----------------------    --------------    
   1         -8.8184068437056499e+02    3.85241567e+02    
   2         -8.9479172163421424e+02    3.52530488e+02    
   3         -8.9964710295121017e+02    3.45355897e+02    
   4         -9.0166434565352631e+02    3.42492001e+02    
   5         -9.0253469216100484e+02    3.41274034e+02    
   6         -9.0291610647739117e+02    3.40743427e+02    
   7         -9.0308438008823282e+02    3.40509925e+02    
   8         -9.0315883800147185e+02    3.40406720e+02    
   9         -9.0319182693941525e+02    3.40361017e+02    
  10         -9.0320645122676478e+02    3.40340760e+02    
  11         -9.0321293594870713e+02    3.40331779e+02    
  12         -9.0321581173683899e+02    3.40327796e+02    
  13         -9.0321708713001851e+02    3.40326030e+02    
  14         -9.0321765277104964e+02    3.40325247e+02    
  15         -9.0321790363715229e+02    3.

In [15]:
problem.grad(U.value)

array([[-1.71145535e+02-1.55323771e+01j, -1.27740822e+01-5.11884173e-01j,
         3.16183780e+01+2.11931414e+01j, ...,
         7.10983905e+01-7.27313334e+00j,  1.85807375e+00+1.52665880e+01j,
         1.75900567e+01+5.01016953e+01j],
       [ 8.37465141e-02-4.50145730e-02j,  1.00347303e+00+9.94449018e-01j,
        -5.16559383e-02-3.45639971e-01j, ...,
        -1.15091520e-02-8.36791131e-01j,  1.02083179e-02-1.05743572e+00j,
        -1.18032504e-01-4.88568245e-01j],
       [-1.54289180e+01-8.25241289e+01j, -8.13444332e+00+5.61672379e+00j,
         1.12608642e+02+1.30788945e+01j, ...,
        -2.29263571e+01+7.99157252e+01j,  2.15953202e+01-4.19741238e+01j,
         3.70063777e+01-3.08818736e+01j],
       ...,
       [ 1.16509544e+01+1.01933975e+01j,  1.87274558e+00+2.71332267e+00j,
         3.00556011e+00-9.53872102e+00j, ...,
        -2.47297452e+01-5.72713446e+00j, -4.11584690e+00+4.98180145e+00j,
        -7.28822716e-01-2.14410735e+01j],
       [-3.42671457e+00-3.83969866e+01j,  1.

In [16]:
problem.grad(solution)

array([[ 1.17920310e+01 -1.4896583j , -1.54627706e+00 -6.53388003j,
        -1.17911011e+01+18.76301105j, ..., -3.57205572e+00 +5.84699634j,
        -1.66100314e+00 -4.76682721j,  1.81957274e+00 +2.1406472j ],
       [-1.73591489e-03 +0.70628808j, -3.93068387e-01 +1.16664277j,
         9.92216435e-01 -2.00513325j, ...,  9.20579773e-01 -1.81620294j,
        -1.58569695e+00 -1.10828943j,  1.26708700e+00 -3.05752599j],
       [ 2.66975080e+00 +3.09286053j,  1.30486346e+00 -7.34720537j,
        -1.03841953e+01-14.6095152j , ..., -6.02815368e+00 +1.11266433j,
        -5.42192035e-01 -2.40068191j, -2.59258486e+00 +4.00524718j],
       ...,
       [-1.82021924e+00 +1.38854913j,  1.66839109e+00 +5.16650611j,
         1.12054859e+01 -5.94148836j, ...,  1.62326929e+00 -4.28419663j,
        -5.43784696e+00 -5.96427359j,  1.99218196e+00 -0.27488584j],
       [ 9.64257894e-02 +8.61983179j,  7.72186838e+00 -4.95564158j,
         7.30714444e+00 +5.40187742j, ...,  8.77543016e-01-15.87368775j,
       

In [11]:
np.linalg.norm(problem.grad(solution))

340.3246225407906

In [12]:
manifold.norm(np.zeros(solution.shape), problem.grad(decompose_psd(U.value)))

457.4888804695002

### Hyperplane rounding

In [13]:
qp_cost = lambda u: -cp.real(cp.quad_form(u, M)).value

In [14]:
complex_hyperplane_rounding(decompose_psd(U.value), qp_cost)

(array([[-1448.79657732]]),
 array([[ 0.14667232-0.98918513j],
        [ 0.06203914+0.99807372j],
        [-0.91406643+0.40556449j],
        [ 0.7863508 +0.61778024j],
        [-0.97874783+0.20506749j],
        [ 0.72367967-0.69013602j],
        [-0.96163793+0.27432187j],
        [-0.96941933+0.24541021j],
        [ 0.4851313 +0.87444132j],
        [-0.99729798-0.07346245j],
        [-0.93464706-0.35557681j],
        [ 0.760043  +0.64987279j],
        [-0.97615265+0.21708524j],
        [ 0.70983466+0.70436834j],
        [ 0.75410175-0.65675761j],
        [ 0.74497213-0.66709559j],
        [ 0.99926171-0.03841925j],
        [ 0.54541664-0.83816507j],
        [ 0.43034871-0.90266272j],
        [-0.06811646+0.99767738j],
        [-0.91643671-0.40017966j],
        [-0.46593752+0.88481762j],
        [-0.99980934-0.01952664j],
        [ 0.41417756-0.91019611j],
        [-0.33445617+0.94241131j],
        [ 0.99786256-0.06534767j],
        [ 0.99991295+0.01319414j],
        [ 0.30354554+0.9528